In [1]:
from astropy.io import fits
from astropy.table import Table
import numpy as np
import pylab as plt
import random
from scipy import stats
from sklearn.neighbors import KDTree
import time
from sklearn.metrics import mean_squared_error
from astropy.cosmology import FlatLambdaCDM
from os import listdir
import desispec
import desispec.io
import speclite.filters
import scipy
from desitarget.targetmask import desi_mask, bgs_mask, mws_mask
from desitarget.cmx.cmx_targetmask import cmx_mask

In [2]:
n=50*10**3
nw=7781

spectra_load=np.load("/global/homes/a/ashodkh/results/spectra.txt.npz")
spectra=spectra_load["arr_0"]
waves_load=np.load("/global/homes/a/ashodkh/results/redshifted_waves.txt.npz")
redshifted_waves=waves_load["arr_0"]
zs_load=np.load("/global/homes/a/ashodkh/results/zs.txt.npz")
zs=zs_load["arr_0"]


# spectra=np.zeros([n,nw])
# redshifted_waves=np.zeros([n,nw])

# with open("/global/homes/a/ashodkh/results/spectra.txt",'r') as f:
#     spectra_str=f.readlines()
# with open("/global/homes/a/ashodkh/results/redshifted_waves.txt",'r') as f:
#     redshifted_waves_str=f.readlines()
    
# for i in range(n):
#     spectra[i,:]=np.array(spectra_str[i].split(","),dtype=float)
#     redshifted_waves[i,:]=np.array(redshifted_waves_str[i].split(","),dtype=float)

In [ ]:
wave_min=np.amin(redshifted_waves,axis=1)
wave_max=np.amax(redshifted_waves,axis=1)

In [4]:
w1=3400
w2=7000

select1=wave_min<w1
select2=wave_max>w2
select=select1*select2
print(len(np.where(select1!=select2)[0]))

5144


In [5]:
wavelength=redshifted_waves[0,:]*(1+zs[0])
print(np.where(wavelength==w1))

(array([], dtype=int64),)


In [ ]:
plt.figure(1)
plt.plot(redshifted_waves[0,:],spectra[0,:])
plt.axvline(x=w1,color='r')
plt.axvline(x=w2,color='r')

d=np.average(redshifted_waves[:,1]-redshifted_waves[:,0])
big_bin=np.arange(w1,w2,d)

spectrum_test=np.interp(big_bin,redshifted_waves[0,:],spectra[0,:])
plt.figure(1)
plt.plot(big_bin,spectrum_test+20)

In [7]:
n_s=len(np.where(select==True)[0])

spectra_s=spectra[select,:]
redshifted_waves_s=redshifted_waves[select,:]

In [8]:
spectra_bin=np.zeros([n_s,len(big_bin)])

for i in range(n_s):
    spectra_bin[i,:]=np.interp(big_bin,redshifted_waves_s[i,:],spectra_s[i,:])

In [10]:
np.savez_compressed("/global/homes/a/ashodkh/results/spectra_bin.txt",spectra_bin)
np.savez_compressed("/global/homes/a/ashodkh/results/big_bin.txt",big_bin)